താഴെ കൊടുത്തിരിക്കുന്ന നോട്ട്‌ബുക്കുകൾ പ്രവർത്തിപ്പിക്കാൻ, നിങ്ങൾ ഇതുവരെ ചെയ്തിട്ടില്ലെങ്കിൽ, .env ഫയലിനുള്ളിൽ `OPENAI_API_KEY` എന്നതായി openai കീ സജ്ജമാക്കേണ്ടതാണ്.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

അടുത്തതായി, നാം എമ്പെഡ്ഡിംഗ് ഇൻഡക്സ് ഒരു പാൻഡാസ് ഡാറ്റാഫ്രെയിമിലേക്ക് ലോഡ് ചെയ്യാൻ പോകുന്നു. എമ്പെഡ്ഡിംഗ് ഇൻഡക്സ് `embedding_index_3m.json` എന്ന JSON ഫയലിൽ സൂക്ഷിച്ചിരിക്കുന്നു. എമ്പെഡ്ഡിംഗ് ഇൻഡക്സ് 2023 ഒക്ടോബർ അവസാനം വരെ ഉള്ള ഓരോ YouTube ട്രാൻസ്ക്രിപ്റ്റിനും എമ്പെഡ്ഡിംഗുകൾ ഉൾക്കൊള്ളുന്നു.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

അടുത്തതായി, ഞങ്ങൾ `get_videos` എന്ന ഫംഗ്ഷൻ സൃഷ്ടിക്കാൻ പോകുന്നു, ഇത് ക്വറിയ്ക്ക് Embedding Index-ൽ തിരയുകയും ചെയ്യും. ഈ ഫംഗ്ഷൻ ക്വറിയുമായി ഏറ്റവും സാമ്യമുള്ള മുകളിൽ 5 വീഡിയോകൾ തിരികെ നൽകും. ഫംഗ്ഷൻ പ്രവർത്തിക്കുന്നത് ഇങ്ങനെ ആണ്:

1. ആദ്യം, Embedding Index-ന്റെ ഒരു പകർപ്പ് സൃഷ്ടിക്കുന്നു.
2. തുടർന്ന്, OpenAI Embedding API ഉപയോഗിച്ച് ക്വറിയുടെ Embedding കണക്കാക്കുന്നു.
3. പിന്നീട്, Embedding Index-ൽ `similarity` എന്ന പുതിയ ഒരു കോളം സൃഷ്ടിക്കുന്നു. `similarity` കോളത്തിൽ ക്വറി Embedding-നും ഓരോ വീഡിയോ സെഗ്മെന്റിന്റെയും Embedding-നും ഇടയിലുള്ള കോസൈൻ സാമ്യം ഉൾപ്പെടുന്നു.
4. തുടർന്ന്, `similarity` കോളം ഉപയോഗിച്ച് Embedding Index ഫിൽട്ടർ ചെയ്യുന്നു. കോസൈൻ സാമ്യം 0.75-ൽ കൂടുതലോ തുല്യമായോ ഉള്ള വീഡിയോകൾ മാത്രം ഉൾപ്പെടുത്താൻ ഫിൽട്ടർ ചെയ്യുന്നു.
5. അവസാനം, `similarity` കോളം അനുസരിച്ച് Embedding Index ക്രമീകരിച്ച് മുകളിൽ 5 വീഡിയോകൾ തിരികെ നൽകുന്നു.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ഈ ഫംഗ്ഷൻ വളരെ ലളിതമാണ്, ഇത് തിരയൽ ക്വറിയുടെ ഫലങ്ങൾ മാത്രം പ്രിന്റ് ചെയ്യുന്നു.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ആദ്യം, Embedding Index ഒരു Pandas Dataframe ആയി ലോഡ് ചെയ്യപ്പെടുന്നു.
2. തുടർന്ന്, ഉപയോക്താവിനെ ഒരു ക്വറി നൽകാൻ പ്രേരിപ്പിക്കുന്നു.
3. പിന്നീട്, `get_videos` ഫംഗ്ഷൻ ക്വറിയ്ക്കായി Embedding Index തിരയാൻ വിളിക്കുന്നു.
4. അവസാനം, ഫലം ഉപയോക്താവിന് പ്രദർശിപ്പിക്കാൻ `display_results` ഫംഗ്ഷൻ വിളിക്കുന്നു.
5. പിന്നീട് ഉപയോക്താവിനെ മറ്റൊരു ക്വറി നൽകാൻ പ്രേരിപ്പിക്കുന്നു. ഉപയോക്താവ് `exit` നൽകുന്നത് വരെ ഈ പ്രക്രിയ തുടരുന്നു.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ml.png)

നിങ്ങൾക്ക് ഒരു ക്വറി നൽകാൻ പ്രേരിപ്പിക്കപ്പെടും. ഒരു ക്വറി നൽകുകയും enter അമർത്തുകയും ചെയ്യുക. ആപ്ലിക്കേഷൻ ക്വറിയുമായി ബന്ധപ്പെട്ട വീഡിയോകളുടെ പട്ടിക തിരികെ നൽകും. ചോദ്യത്തിന് ഉത്തരം ലഭിക്കുന്ന വീഡിയോയിലെ സ്ഥലം കാണിക്കുന്ന ഒരു ലിങ്കും ആപ്ലിക്കേഷൻ നൽകും.

പരീക്ഷിക്കാൻ ചില ക്വറികൾ ഇവിടെ കൊടുത്തിരിക്കുന്നു:

- Azure Machine Learning എന്താണ്?
- Convolutional neural networks എങ്ങനെ പ്രവർത്തിക്കുന്നു?
- Neural network എന്താണ്?
- Azure Machine Learning ഉപയോഗിച്ച് Jupyter Notebooks ഉപയോഗിക്കാമോ?
- ONNX എന്താണ്?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
